## Final Project
### Data Science LHL Project: Restaurant Reccomenation system

### Part 1: Data Collection

By: Chloe Phuong

In [197]:
import numpy as np
import pandas as pd
import requests

#### Request API for Bike Station

In [212]:
# CityBikes

# API endpoint for Toronto city
url = 'https://api.citybik.es/v2/networks/toronto'

# Send GET request to retrieve bike-sharing data for Toronto
response = requests.get(url)

# Extract the data from the response
data = response.json()

# Extract the list of stations
stations = data['network']['stations']

# Iterate over each station and access the details
for i, station in enumerate(stations):
    if i > 100:
        break  # Exit the loop if the limit is reached
    
    station_id = station.get('id')
    name = station.get('name')
    latitude = station.get('latitude')
    longitude = station.get('longitude')

# Create a list to store the station details
station_details = []

# Iterate over each station and access the details
for i, station in enumerate(stations):
    if i > 100:
        break  # Exit the loop if the limit is reached
    
    station_id = station['id']
    name = station['name']
    latitude = station['latitude']
    longitude = station['longitude']

    # Append the station details as a dictionary to the list
    station_details.append({
        'Station ID': station_id,
        'Name': name,
        'Latitude': latitude,
        'Longitude': longitude,
    })

# Create the bike stations DataFrame
bike_df = pd.DataFrame(station_details)
bike_df

,Station ID,Name,Latitude,Longitude
0,fb337bbed72e2be090071e199899b2be,Queen St E / Woodward Ave,43.665269,-79.319796
1,4ff88d5880e71aa40d34cfe5d09b0ca7,Primrose Ave / Davenport Rd,43.671420,-79.445947
2,a09c67c0b419654d907c9134b108e328,Queen St. E / Rhodes Ave.,43.666224,-79.317693
3,d6a9daee68070a8b106cfb598d81308c,Bond St / Queen St E,43.653236,-79.376716
4,8f8af40d9388c8a3962559e8681d3db7,Church St / Alexander St,43.663722,-79.380288
...,...,...,...,...
96,908b17c0d06011373777bace0a3cd74c,Beltline Trail / Chaplin Cres.,43.704461,-79.423122
97,d2ce6ae839b7df2c912ab140253a5574,Lake Shore Blvd E / Leslie St,43.659195,-79.329196
98,1d35b8fd44d73aabbb39f6e142ee3e7e,King St E / Berkeley St,43.651885,-79.364900
99,72de8a8e0a0966768f8115d755532e4c,Rosedale Park (20 Scholfield Ave),43.682645,-79.377557


#### Request Yelp API

In [216]:
API_key = 'IVOE8RKoC3-F0Vqm4AjugMU-vCOf26cA_gDvbDSh35Ly8-qbO5FZBfuOkDbcQunkn1g2k3hEArtEPUbkL6qrJoJZASz0sUUTpfiCgLj2XhnR7JxSjfQmcBO8RU6qZHYx'
url = 'https://api.yelp.com/v3/businesses/search'
radius = 1000  # Radius in meters
limit = 10

In [226]:
# Create an empty list to store the extracted details
rest_details = []

# Iterate over each row in the dataframe
for index, row in bike_df.iterrows():
    station_id = row['Station ID']
    latitude = row['Latitude']
    longitude = row['Longitude']

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': radius,
        'term': 'restaurants',
        'limit': limit
    }
    headers = {
        'Authorization': f'Bearer {API_key}'
    }

    # Send API request to Yelp
    response = requests.get(url, params=params, headers=headers)
    rest_data = response.json()

    # Check if 'businesses' key exists in the response
    if 'businesses' in rest_data:
        businesses = rest_data['businesses']

        # Iterate over each business and extract the details
        for business in businesses:
            business_id = business['id']
            name = business['name']
            address = ', '.join(business['location']['display_address'])
            city = business['location']['city']
            postal_code = business['location']['zip_code']
            latitude = business['coordinates']['latitude']
            longitude = business['coordinates']['longitude']
            stars = business['rating']
            review_count = business['review_count']
            is_open = not business['is_closed']
            categories = business['categories']
            hours = business.get('hours')

            # Append the restaurant details to the list
            rest_details.append({
                'business_id': business_id,
                'name': name,
                'address': address,
                'city': city,
                'postal_code': postal_code,
                'latitude': latitude,
                'longitude': longitude,
                'stars': stars,
                'review_count': review_count,
                'is_open': is_open,
                'categories': categories,
                'hours': hours
            })

# Create a DataFrame from the list of restaurant details
rest_df = pd.DataFrame(rest_details)

# Print the DataFrame
rest_df

,business_id,name,address,city,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
0,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.666270,-79.318020,4.5,15,True,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",None
1,Huo1lJmVkdqvFuLtA74Nhw,Casa Di Giorgios,"1646 Queen St E, Toronto, ON M4L 1G3, Canada",Toronto,M4L 1G3,43.666850,-79.315180,4.0,75,True,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",None
2,yP71_LfUPnGIRqA2g09RoQ,Lake Inez,"1471 Gerrard Street E, Toronto, ON M4L 2A1, Ca...",Toronto,M4L 2A1,43.672340,-79.320640,4.5,94,True,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",None
3,_FIJ1fCMlFAQ8uzxv6AIfg,Maru Japanese Bistro,"1402 Queen Street E, Toronto, ON M4L 1C7, Canada",Toronto,M4L 1C7,43.664720,-79.324263,4.0,17,True,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",None
4,0iTpgvz5XJzFAo253nIE0g,The Sidekick,"1374 Queen Street E, Toronto, ON M4L 1C9, Canada",Toronto,M4L 1C9,43.664589,-79.325111,4.5,31,True,"[{'alias': 'cafes', 'title': 'Cafes'}]",None
...,...,...,...,...,...,...,...,...,...,...,...,...
999,YHubCF0Ua0uC6HC2tm-arQ,The Federal,"1438 Dundas Street W, Toronto, ON M6J 3L5, Canada",Toronto,M6J 3L5,43.649770,-79.430110,4.0,173,True,"[{'alias': 'breakfast_brunch', 'title': 'Break...",None
1000,HHvO4MgBFabsmlInou6viQ,Le Baratin,"1600 Dundas Street. W, Toronto, ON M6K 1T8, Ca...",Toronto,M6K 1T8,43.650070,-79.435100,4.5,130,True,"[{'alias': 'french', 'title': 'French'}]",None
1001,xIKmVF_kRG_NDsmg0xmzTw,Nuna Kitchen & Bar,"1265 Queen Street W, Toronto, ON M6K 1L4, Canada",Toronto,M6K 1L4,43.641540,-79.431120,4.5,19,True,"[{'alias': 'international', 'title': 'Internat...",None
1002,6hGEIpALyUhte4grCY8Hog,The Opium Bar,"1321 Dundas Street West, Toronto, ON M6J 1X8, ...",Toronto,M6J 1X8,43.649280,-79.426280,4.5,20,True,"[{'alias': 'whiskeybars', 'title': 'Whiskey Ba...",None


In [227]:
rest_df.shape

(1004, 12)